In [33]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import my_lib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
pipeline = my_lib.Pipeline()
raw_df = pipeline.load_data()
raw_df = pipeline.encode_to_category(raw_df)
raw_df = pipeline.tokenize(raw_df)
raw_df.head()

,tweet,label,enterprise,is_irrelevant,token_list,clean_tweet,number_vector,input_vector
0,20 min line @apple store @short pump.,1,0,0,20 min line @apple store @short pump.,20 min line @apple store @short pump .,"[8, 240, 306, 439, 179, 462, 306, 23, 179, 462...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,Nueva tecnología convierte cualquier superfici...,3,1,1,nueva tecnología convierte cualquier superfici...,nueva tecnología convierte cualquier superfici...,"[462, 444, 324, 6, 307, 306, 601, 324, 533, 46...","[0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Some people should not post replies in #Google...,1,2,0,some people should not post replies in #google...,some people should not post replies in #google...,"[523, 373, 439, 324, 306, 629, 324, 373, 629, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,I know a few others having same issue RT @Joel...,2,0,0,i know a few others having same issue rt @joel...,i know a few others having same issue rt @joel...,"[179, 306, 424, 462, 373, 85, 306, 307, 306, 1...","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"#Microsoft - We put the """"backwards"""" into bac...",2,1,0,"#microsoft - we put the """"backwards"""" into bac...","#microsoft - we put the "" "" backwards "" "" ...","[294, 439, 179, 533, 354, 373, 523, 373, 162, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [35]:
df = raw_df.loc[raw_df.label.isin([0,2])]
df = my_lib.processing.remove_useless_col(df, keep_col_list=["label", "clean_tweet"])
df_train_data, df_test_data = my_lib.processing.split_data(df)
print(f"SHAPE -> train : {df_train_data.shape}, test : {df_test_data.shape}")
df.head()

SHAPE -> train : (730, 2), test : (183, 2)


,label,clean_tweet
3,2,i know a few others having same issue rt @joel...
4,2,"#microsoft - we put the "" "" backwards "" "" ..."
5,2,#twitter is sooo trash ritenow with all dezz #...
11,2,i gave a try to windows 8 dev . preview and i...
14,0,@apple . . good ipad


In [36]:
bert = my_lib.model.BERT()
bert.score(df_train_data["clean_tweet"].values, df_train_data["label"].values)


Some weights of the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_199']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0100200300400500600700

0.7534246575342466